# Define Business Goal

In this project, you will use a machine learning model to predict passenger survival on the Titanic.

# Get Data & Exploring Titanic Dataset

I try to understand the data better in order to adjust my feature engineering in the Training section of this Notebook

In [4]:
cd ..

/Users/oguzhanbekar/OneDrive/Spiced/euclidean-eukalyptus-student-code/02_week


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import KBinsDiscretizer,FunctionTransformer,MinMaxScaler,OneHotEncoder,StandardScaler
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier  
from sklearn.metrics import roc_curve, roc_auc_score

In [5]:
df = pd.read_csv('data/train.csv',index_col=0)

In [6]:
del df['Ticket']

In [7]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,NaN,S


In [8]:
len(df[(df['Pclass']==1)&(df['Survived']==1)])/len(df[df['Pclass']==1])

0.6296296296296297

In [ ]:
sns.countplot(x=df['Pclass'],hue=df['Sex'])

In [ ]:
sns.histplot(x=df['Fare'],bins=25)

In [ ]:
df['SibSp'].value_counts()

In [ ]:
df[['SibSp', 'Survived']].groupby('SibSp', as_index = False).mean().sort_values(by = 'Survived', ascending = False)


In [ ]:
sns.heatmap(data=df[['Survived','Age','SibSp','Fare']].corr(),annot = True, fmt = '.2f', cmap = 'coolwarm')

In [ ]:
# Create a table counting the number of surviving/dead passengers separately for 1st/2nd/3rd class and male/female.
grouped_class = df.groupby(['Survived','Pclass']).size().reset_index(name='counts')



In [ ]:
df['Cabin']

In [ ]:
sns.pairplot(df)

### Will do this calculation later

In [9]:
cols = df['Cabin'].str

In [10]:
b = '2'

In [ ]:
isinstance(int(b), int) # True


In [ ]:
i = 0
row = []

for i in cols[i]:
    if isinstance(int(cols[i]), str):
        
    i += 1

In [ ]:
df.head()

In [ ]:
sns.countplot(data=df,x='cabin_two',hue='Survived')

In [ ]:
sns.histplot(data=df,x='Age',y='Survived',bins=10)

In [ ]:
df['Cabin'].nunique

# Define X and y

### With company or not 

In [11]:
df['SibParch'] = df['SibSp'] + df['Parch']

In [12]:
df['SibParch'].unique()

array([ 1,  0,  4,  2,  6,  5,  3,  7, 10])

In [13]:
df['SibParch'] = df['SibParch'].replace([2],[1])
df['SibParch'] = df['SibParch'].replace([3],[1])
df['SibParch'] = df['SibParch'].replace([4],[1])
df['SibParch'] = df['SibParch'].replace([5],[1])
df['SibParch'] = df['SibParch'].replace([6],[1])
df['SibParch'] = df['SibParch'].replace([7],[1])
df['SibParch'] = df['SibParch'].replace([10],[1])


### Cabin Letter creation

In [14]:
df['cabin_letter'] = df['Cabin'].str[0]

In [15]:
del df['Cabin']

In [16]:
df['cabin_letter'].isnull().sum()

687

In [17]:
del df['Parch']

In [18]:
type(df.iloc[3,3])

str

In [19]:
type(df.iloc[3,-2])

numpy.int64

In [20]:
# split to X, and y
X = df.iloc[:,1:]
y = df['Survived']



In [21]:
mode = df['Embarked'].dropna().mode()[0]
df['Embarked'].fillna(mode, inplace = True)


### Transformer

In [22]:
# feature engineering for numericals
age_feature = ["Age"]

# create a sequential pipeline 
# output of one step will be input to the next 
age_transformer = make_pipeline(
    SimpleImputer(strategy="mean"), 
    StandardScaler()    
)

# feature engineering for numericals
fare_feature = [ "Fare"]

# create a sequential pipeline 
# output of one step will be input to the next 
fare_transformer = make_pipeline(
    SimpleImputer(strategy="mean"), 
    MinMaxScaler()
    )

In [23]:
# feature engineering for categorical
categorical_features = ["Embarked", "Sex", "Pclass", "cabin_letter","SibParch"]

# handle unknown means it will ignore nan's if it finds them
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

In [24]:
def name_length(df):
    length = df[df.columns[0]].str.len()
    return length.values.reshape(-1, 1)


In [25]:
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Fare,Embarked,SibParch,cabin_letter
PassengerId,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,7.2500,S,1,NaN
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,71.2833,C,1,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,7.9250,S,0,NaN
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,53.1000,S,1,C
5,0,3,"Allen, Mr. William Henry",male,35.0,0,8.0500,S,0,NaN


In [26]:
preprocessor = ColumnTransformer(
    transformers=[
        ("age", age_transformer, age_feature),
        ("fare", fare_transformer, fare_feature),
        ("cat", categorical_transformer, categorical_features),
        ('name', FunctionTransformer(name_length), ['Name']),
#         ('do nothing','passthrough',['Name'])
    ],
    
)


### Pipelines

In [27]:
# create the model pipeline
pipeline = make_pipeline(preprocessor, LogisticRegression(max_iter=300))
pipeline_rf = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=70,max_depth=2))

### Split train and test data

In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [29]:
# fit the pipeline to training data
pipeline.fit(X_train, y_train)
pipeline_rf.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('age',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['Age']),
                                                 ('fare',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('minmaxscaler',
                                                                   MinMaxScaler())]),
                                                  ['Fare']),
                                              

In [30]:
pipeline.score(X_train,y_train),pipeline_rf.score(X_train,y_train)

(0.8188622754491018, 0.7799401197604791)

In [31]:
# calculate the accuracy score from training data
print("LogReg model score: %.3f" % pipeline.score(X_train, y_train))
print("model score: %.3f" % pipeline_rf.score(X_train, y_train))

# calculate the accuracy score from test data
print("model score: %.3f" % pipeline.score(X_test, y_test))
print("model score: %.3f" % pipeline_rf.score(X_test, y_test))

# get predictions from the pipeline
print(pipeline.predict(X_test))
print(pipeline_rf.predict(X_test))

# get prediction probabilities from the pipeline 
print(pipeline.predict_proba(X_test)[:, 1])
print(pipeline_rf.predict_proba(X_test)[:, 1])
y_score1 = pipeline.predict_proba(X_test)[:, 1]
y_score2 = pipeline_rf.predict_proba(X_test)[:, 1]


LogReg model score: 0.819
model score: 0.780
model score: 0.803
model score: 0.731
[0 1 1 0 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0
 0 1 0 0 1 1 1 0 1 0 1 1 0 1 1 1 1 0 1 0 0 1 0 1 0 1 1 0 1 1 0 0 1 1 1 0 1
 0 0 0 1 0 1 1 1 1 1 0 1 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 1 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0
 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1
 1 0 1 0 0 0 1 1 1 1 1 1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 0 0
 0]
[0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1
 0 1 0 0 1 0 0 0 1 0 1 1 0 1 1 1 1 1 1 0 0 1 0 0 0 1 1 0 1 1 1 0 1 1 0 0 1
 0 0 0 1 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 1 1 0
 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1
 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0 0 0 0 0
 0]
[0.066101

In [32]:
X_train.head()

,Pclass,Name,Sex,Age,SibSp,Fare,Embarked,SibParch,cabin_letter
PassengerId,,,,,,,,,
611,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,1,31.275,S,1,NaN
749,1,"Marvin, Mr. Daniel Warner",male,19.0,1,53.100,S,1,D
62,1,"Icard, Miss. Amelie",female,38.0,0,80.000,NaN,0,B
378,1,"Widener, Mr. Harry Elkins",male,27.0,0,211.500,C,1,C
523,3,"Lahoud, Mr. Sarkis",male,NaN,0,7.225,C,0,NaN


### Preparing Predictions Test Data

In [33]:
df_test = pd.read_csv('data/test.csv')

In [34]:
df_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [35]:
df_test['SibParch'] = df_test['SibSp'] + df_test['Parch']

df_test['SibParch'].unique()

array([ 0,  1,  2,  4,  3,  5,  7,  6, 10])

In [36]:
df_test['SibParch'] = df_test['SibParch'].replace([2],[1])
df_test['SibParch'] = df_test['SibParch'].replace([3],[1])
df_test['SibParch'] = df_test['SibParch'].replace([4],[1])
df_test['SibParch'] = df_test['SibParch'].replace([5],[1])
df_test['SibParch'] = df_test['SibParch'].replace([6],[1])
df_test['SibParch'] = df_test['SibParch'].replace([7],[1])
df_test['SibParch'] = df_test['SibParch'].replace([10],[1])


In [37]:
df_test['cabin_letter'] = df_test['Cabin'].str[0]

In [38]:
df_test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked', 'SibParch', 'cabin_letter'],
      dtype='object')

In [39]:
X_train.columns

Index(['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked', 'SibParch',
       'cabin_letter'],
      dtype='object')

In [40]:
df_test = df_test.set_index('PassengerId')

In [41]:
df_test.drop(['Ticket','Parch'],axis=1,inplace=True)

In [42]:
column_names = ['Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Fare', 'Embarked', 'SibParch',
       'cabin_letter']

df_test = df_test.reindex(columns=column_names)

In [43]:
X_train.head()

,Pclass,Name,Sex,Age,SibSp,Fare,Embarked,SibParch,cabin_letter
PassengerId,,,,,,,,,
611,3,"Andersson, Mrs. Anders Johan (Alfrida Konstant...",female,39.0,1,31.275,S,1,NaN
749,1,"Marvin, Mr. Daniel Warner",male,19.0,1,53.100,S,1,D
62,1,"Icard, Miss. Amelie",female,38.0,0,80.000,NaN,0,B
378,1,"Widener, Mr. Harry Elkins",male,27.0,0,211.500,C,1,C
523,3,"Lahoud, Mr. Sarkis",male,NaN,0,7.225,C,0,NaN


In [44]:
df_test.head()

,Pclass,Name,Sex,Age,SibSp,Fare,Embarked,SibParch,cabin_letter
PassengerId,,,,,,,,,
892,3,"Kelly, Mr. James",male,34.5,0,7.8292,Q,0,NaN
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,7.0000,S,1,NaN
894,2,"Myles, Mr. Thomas Francis",male,62.0,0,9.6875,Q,0,NaN
895,3,"Wirz, Mr. Albert",male,27.0,0,8.6625,S,0,NaN
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,12.2875,S,1,NaN


## Predictions

In [45]:
predictions = pipeline.predict(df_test)

In [46]:
predictions_rf = pipeline_rf.predict(df_test)

In [47]:
output = pd.DataFrame({'PassengerID' : df_test.index, 'Survived' : predictions})

In [48]:
output['Survived'] = output['Survived'].astype(int)
output.to_csv('final.csv', index=False)

In [49]:
output_rf = pd.DataFrame({'PassengerID' : df_test.index, 'Survived' : predictions_rf})

In [50]:
output_rf['Survived'] = output['Survived'].astype(int)
output_rf.to_csv('final_rf.csv', index=False)